# Introduction to Machine Learning

This practical uses scikit-learn to make classification schemes for proteins based on metadata supplied in DUDE-E.features.10.tgz. The first part of the code just extracts files into the appropriate arrays.

If you would like to explore other datasets, simply modify the combined_data function call in cell 4.

Simon & Itai Dec 2019

In [1]:
import sklearn
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import SGDClassifier
import numpy as np
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def list_dir():      
    Active = []
    Decoy = []
    active_list=sorted(os.listdir("/home/sabs-r3/Documents/Data-science/ML/Active"))
    decoy_list = sorted(os.listdir("/home/sabs-r3/Documents/Data-science/ML/Decoy"))
    print(active_list)
    for i in active_list:
        Active.append(i)
    for i in decoy_list:
        Decoy.append(i)      
    return Active, Decoy


def combined_data(index):
    if index >0 or index<9:
        active_list = list_dir()[0]
        os.chdir('/home/sabs-r3/Documents/Data-science/ML/Active')
        active_data = pd.read_csv(active_list[index])
        active_data["Activity"] = [1]*len(active_data.index)
        os.chdir('..')
    
        decoy_list = list_dir()[1]
        os.chdir('/home/sabs-r3/Documents/Data-science/ML/Decoy')
        decoy_data = pd.read_csv(decoy_list[index])
        decoy_data["Activity"] = [0]*len(decoy_data.index)
        os.chdir('..')
        data = pd.concat([active_data, decoy_data])
    else:
        print('Enter an index between 0 and 9')
    return data


# 

In [ ]:
# extract data & split into training & testing sets
data = combined_data(0) #modify this for other data!
y = data['Activity'].to_numpy()
X = data.to_numpy()[:, 1:-1]
training_data, test_data, training_classes, test_classes = train_test_split(X, y, random_state=0)

The _predict_ function fits training data to a given classification scheme and immediately predicts classes in test data. This first one fits a random forest classification scheme. Comparison of the predicted to actual classes for the test data give a quick heuristic for function.

In [ ]:
def predict(training_data, training_classes, test_data, clf):
    clf.fit(training_data, training_classes)
    predicted_classes = clf.predict(test_data)
    return predicted_classes

In [ ]:
pred_classes = predict(training_data, 
                       training_classes, 
                       test_data, 
                       RandomForestClassifier(random_state=0, n_estimators=500))

In [ ]:
# Simple test that it works
matches = 0
for n in range(len(pred_classes)):
    if pred_classes[n] == test_classes[n]:
        matches = matches + 1
print("This matches %.2f percent of records" % (matches*100 / len(pred_classes)))

## Testing the prediction

Receiver Operator Characteristic curves are a way of showing the true positive rate (tpr) versus the false positive rate (fpr) for a method. The area under this curve is a representation of how well the function works to minimise false positives and negatives.

The confusion matrix is another way of visualising this: the diagonal top left to bottom right shows the numbers of points that are accurately classified (true positives and negatives). Conversely the opposite diagonal shows numbers of wrongly classified points.

In [ ]:
def plot_it(test_classes, pred_classes, title):
    fpr, tpr, _ = roc_curve(test_classes, pred_classes)
    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1], 'o--')
    plt.xlim([-0.01,1.0])
    plt.ylim([0.0,1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for '+title)
    plt.show()
    print('Area Under Curve is %.3f' % auc(fpr, tpr))
    print('Confusion matrix is:')
    print(confusion_matrix(test_classes, pred_classes))

In [ ]:
plot_it(test_classes, pred_classes, "Random Forest")

## Trying other classification schemes

The decision tree classifier is supposed to be a simpler version of the random forest (which takes multiple decision trees), but appears to work better on this data. We don't know why yet.

In [ ]:
pred_classes = predict(training_data, 
                            training_classes, 
                            test_data, 
                            DecisionTreeClassifier(random_state=0))
plot_it(test_classes, pred_classes, "Decision Tree")

## Testing with other datasets
To prove that it's not this one dataset being strange, we tried it on others below.

In [ ]:
# extract data & split into training & testing sets
data = combined_data(2) #modify this for other data!
y = data['Activity'].to_numpy()
X = data.to_numpy()[:, 1:-1]
training_data, test_data, training_classes, test_classes = train_test_split(X, y, random_state=0)

pred_classes = predict(training_data, 
                       training_classes, 
                       test_data, 
                       RandomForestClassifier(random_state=0, n_estimators=500))
plot_it(test_classes, pred_classes, "Random forest (dataset 2)")

In [ ]:
pred_classes = predict(training_data, 
                       training_classes, 
                       test_data, 
                       DecisionTreeClassifier(random_state=0))
plot_it(test_classes, pred_classes, "Decision tree (dataset 2)")